<center><img src="https://github.com/randall-romero/EC4301/blob/master/Laboratorios/Tema09-SEM/escuela-de-economia.png?raw=1" width="260" height="60"></center>
<center>
    <b>EC4301 MACROECONOMETRÍA</b><br>
    <b>Profesor:  Randall Romero Aguilar, PhD</b>
    <br><b>Estudiantes: David Mora Salazar y Manfred Ramírez Alfaro</b><br>
<br><br>
<b>Tarea 8:</b>
<div style="font-size:175%;color:white; background-color: #0064b0;">Tarea 8: Modelos de ecuaciones simultáneas (SEM)</div>
<div style="font-size:250%;color:white; background-color: #0064b0;">El modelo de la telaraña</div> 
</center>

<h2>Pregunta 1</h2>

# Objetivo

En este cuaderno se presenta el modelo de la telaraña para dos mercados, con el fin de ilustrar los siguientes conceptos
* Sesgo de simultaneidad
* Estimación de mínimos cuadrados en dos etapas
* Efectos dinámicos de una perturbación

En particular, si $q$ y $p$ denotan cantidad y precio,$s$ y $d$ representan oferta y demanda, $m$ y $w$ maíz y trigo, y $t$ tiempo, el modelo es

\begin{align*} 
q^s_{mt} &= 0.5p_{m,t-1} - 0.2p_{w,t-1} + 200 + \epsilon^s_{mt}  \tag{oferta de maíz}\\
q^s_{wt} &= -0.2p_{m,t-1} + 0.4p_{w,t-1} + 80 + \epsilon^s_{wt}  \tag{oferta de trigo} \\
q^d_{mt} &= - 0.2p_{mt} + 1.2p_{wt} + 100 + \epsilon^d_{mt} \tag{demanda de maíz}\\
q^d_{wt} &= 1.1p_{mt} - 0.4p_{wt} + 50 + \epsilon^d_{wt} \tag{demanda de trigo}
\end{align*}

y puede escribirse
$$
\underset{\Gamma}{\begin{pmatrix}1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 1 & 0 & 0.2 & -1.2 \\ 0 & 1 & -1.1 & 0.4\end{pmatrix}} 
\underset{Y_t}{\begin{pmatrix}q_{mt} \\ q_{wt} \\ p_{mt} \\ p_{wt}\end{pmatrix}} = 
\underset{B}{\begin{pmatrix}0.5 & -0.2 \\ -0.2 & 0.4 \\  0 & 0 \\ 0 & 0\end{pmatrix}} 
\underset{Y_{t-1}}{\begin{pmatrix}p_{m,t-1} \\ p_{w,t-1}\end{pmatrix}}  +
\underset{c}{\begin{pmatrix}200 \\ 80 \\ 100 \\50\end{pmatrix}} +
\underset{\epsilon_t}{\begin{pmatrix}\epsilon^s_{mt} \\ \epsilon^s_{wt} \\ \epsilon^d_{mt} \\ \epsilon^d_{wt}\end{pmatrix}}
$$

# Preparación

Importamos los paquetes de Python necesarios. En particular
* numpy: para operaciones de álgebra lineal
* pyplot: para graficar
* pandas: para manejar datos en una tabla
* statstmodels: estimaciones econométricas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.gmm import IV2SLS
from sklearn.metrics import mean_squared_error

%matplotlib inline
sns.set(style="white")
np.set_printoptions(precision=3)

Definimos los parámetros del modelo

In [2]:
T = 100
endogenas = ['Q_maiz','Q_trigo','P_maiz','P_trigo']

Gamma = np.array([      # RELACIÓN SIMULTANEA
    [1, 0,   0,    0],  # oferta maiz
    [0, 1,   0,    0],  # oferta trigo
    [1, 0, 0.2, -1.2],  # demanda maiz
    [0, 1,-1.1, 0.4]])  # demanda trigo 

cstr = np.array([200,80,100,50])  # interceptos forma estructural

Beta = np.array([  # PENDIENTES ESTRUCTURALES
    [ 0.5, -0.2],  # oferta maiz
    [-0.2,  0.4],  # oferta trigo
    [   0,    0],  # demanda maiz
    [   0,    0]]) # demanda trigo 

A partir de la inversa de $\Gamma$, obtenemos los parámetros de la forma reducida

In [3]:
Gammainv = np.linalg.inv(Gamma)

cred = Gammainv @ cstr # interceptos forma reducida
Pi = Gammainv @ Beta   # pendientes forma reducida 

In [4]:
Pi

array([[ 0.5  , -0.2  ],
       [-0.2  ,  0.4  ],
       [-0.032,  0.323],
       [ 0.411, -0.113]])

## Identificación del sistema
Ahora verificamos que el sistema está exactamente identificado, utilizando las condiciones de rango y de orden.

In [5]:
coeffs = np.c_[Gamma, Beta, cstr]
restr = np.abs(coeffs) == coeffs**2 # coeficientes restringidos son 1, -1 o 0. Esto funciona para este ejemplo en particular
for i, fila in enumerate(restr):
    temp = coeffs[:,fila]
    print(f'\nLa ecuación {i+1} tiene {temp.shape[1]} restricciones y sus eigenvalores son', np.linalg.eigvals(temp))
    


La ecuación 1 tiene 4 restricciones y sus eigenvalores son [-0.853  1.453  1.     1.   ]

La ecuación 2 tiene 4 restricciones y sus eigenvalores son [-0.853  1.453  1.     1.   ]

La ecuación 3 tiene 4 restricciones y sus eigenvalores son [ 1.452  1.203 -0.452 -0.203]

La ecuación 4 tiene 4 restricciones y sus eigenvalores son [ 1.452  1.203 -0.452 -0.203]


# Simulación del modelo

Definimos la función *telaraña*, la cual simula $T$ observaciones de los precios y cantidades de trigo, luego de desechar las primeras *drop* observaciones. El resultado se retorna como una tabla de *Pandas*, la cual facilita la creación de gráficos y la estimación econométrica posterior.

In [6]:
def telaraña(T, drop=10, estocastico=True):
    if estocastico:
        e_struc = np.random.randn(T+drop,4)
        e_reduc = e_struc @ Gammainv.T
    else:
        e_reduc = np.zeros((T+drop,4))
            
    y = np.zeros((T+drop,4))
    y[0,-2:] = [120, 70]
    for t in range(T+drop-1):
        y[t+1] = cred + Pi @ y[t,-2:] + e_reduc[t+1]
    return pd.DataFrame(y[drop:], columns=endogenas)

Los datos simulados aparentan ser estacionarios. Para confirmarlo, podemos escribir el modelo en forma de un VAR(1) reducido:
$$
\begin{pmatrix}1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 1 & 0 & 0.2 & -1.2 \\ 0 & 1 & -1.1 & 0.4\end{pmatrix}
\begin{pmatrix}q_{mt} \\ q_{wt} \\ p_{mt} \\ p_{wt}\end{pmatrix} = 
\begin{pmatrix}0 & 0 & 0.5 & -0.2 \\0 & 0 &  -0.2 & 0.4 \\0 & 0 &   0 & 0 \\0 & 0 &  0 & 0\end{pmatrix}
\begin{pmatrix}q_{m,t-1} \\ q_{w,t-1} \\ p_{m,t-1} \\ p_{w,t-1}\end{pmatrix}  +
\begin{pmatrix}200 \\ 80 \\ 100 \\50\end{pmatrix} +
\begin{pmatrix}\epsilon^s_{mt} \\ \epsilon^s_{wt} \\ \epsilon^d_{mt} \\ \epsilon^d_{wt}\end{pmatrix}
$$

y obtenemos los eigenvalores de su matriz de coeficientes autorregresivos.




In [7]:
arcoefstr = np.c_[np.zeros([4,2]),Beta]
arcoef = Gammainv @ arcoefstr
print('La matriz de coeficientes autorregresivos en forma reducida es \n', arcoef)
print('\nLos eigenvalores de la matriz autorregresiva en forma reducida son ', np.linalg.eigvals(arcoef) )

La matriz de coeficientes autorregresivos en forma reducida es 
 [[ 0.     0.     0.5   -0.2  ]
 [ 0.     0.    -0.2    0.4  ]
 [ 0.     0.    -0.032  0.323]
 [ 0.     0.     0.411 -0.113]]

Los eigenvalores de la matriz autorregresiva en forma reducida son  [ 0.     0.     0.294 -0.439]


Como todos los eigenvalores están dentro del círculo unitario, hemos confirmado que el sistema es estable y por tanto estacionario. Siendo estacionario, el sistema tiene media constante, que calculamos a continuación:

In [8]:
esperados = np.linalg.inv(np.eye(4) - arcoef) @ cred
for val in zip(endogenas, esperados): print('%10s: %6.2f' % val)

    Q_maiz: 224.44
   Q_trigo: 108.57
    P_maiz:  96.83
   P_trigo: 119.84


## Estimación OLS de la forma estructural del modelo

A continuación simulamos el modelo de la telaraña 1000 veces, y estimamos cada ecuación del modelo con OLS.

In [9]:
NSIMUL = 1000

def ols_params(modelo, T=24):
    datos = telaraña(T)
    datos['LP_maiz'] = datos['P_maiz'].shift()   # precio maiz rezagado
    datos['LP_trigo'] = datos['P_trigo'].shift() # precio trigo rezagado
    return ols(modelo, datos).fit().params

true_params = {
    'ms': np.r_[cstr[0],Beta[0]],
    'ws': np.r_[cstr[1],Beta[1]],
    'md': np.r_[cstr[2], -Gamma[2,-2:]],
    'wd': np.r_[cstr[3], -Gamma[3,-2:]]}

description = {
    'ms': 'Oferta de maíz',
    'ws': 'Oferta de trigo',
    'md': 'Demanda de maíz',
    'wd': 'Demanda de trigo'}

<b>Estimación del error cuadrático medio del estimador MCO</b>

Error cuadrático medio de la demanda de maíz.

In [10]:
simul = pd.concat([ols_params('Q_maiz ~ 1 + P_maiz + P_trigo', 24) for x in range(NSIMUL)],axis=1).T

In [12]:
def error_cuadratico_medio(simulacion,observado,parametro):
    real = []
    predic = []
    for i in range(1000):
        real.append(observado)
        predic.append(simulacion[parametro][i])
    return mean_squared_error(real, predic)

Error cuadrático medio del coeficiente estimado del precio del maíz.

In [13]:
error_cuadratico_medio(simul,-0.2,"P_maiz")

0.01715317254961159

Error cuadrático medio del coeficiente estimado del precio del trigo.

In [14]:
error_cuadratico_medio(simul,1.2,"P_trigo")

0.2451547131352129

Error cuadrático medio del intercepto de la demanda de maíz.

In [38]:
error_cuadratico_medio(simul,100,"Intercept")

3039.427218985352

Error cuadrático medio de la demanda de trigo.

In [15]:
simulT = pd.concat([ols_params('Q_trigo ~ 1 + P_maiz + P_trigo', 24) for x in range(NSIMUL)],axis=1).T

Error cuadrático medio del coeficiente estimado del precio del maíz.

In [17]:
error_cuadratico_medio(simulT,1.1,"P_maiz")

0.24824732027797977

Error cuadrático medio del coeficiente estimado del precio del trigo.

In [18]:
error_cuadratico_medio(simulT,-0.4,"P_trigo")

0.03857269515918955

Error cuadrático medio del intercepto de la demanda de trigo.

In [36]:
error_cuadratico_medio(simulT,50,"Intercept")

1122.9055838162578

## Estimación 2SLS de la forma estructural del modelo

In [20]:
def tsls_params(modelo, T=24):
    datos = telaraña(T)
    datos['LP_maiz'] = datos['P_maiz'].shift()   # precio maiz rezagado
    datos['LP_trigo'] = datos['P_trigo'].shift() # precio trigo rezagado
 
    etapa1 = pd.concat([ols('%s ~ LP_maiz + LP_trigo' % x, datos).fit().fittedvalues for x in endogenas],axis=1)
    etapa1.columns = [st + '_OLS' for st in endogenas]
    datos = pd.concat([datos, etapa1], axis=1)
    return ols(modelo, datos).fit().params


Luego de realizar esto, se requiere calcular el error cuadrático medio para el estimador MC2E.

Error cuadrático medio de la demanda de maíz.

In [22]:
simulM = pd.concat([tsls_params('Q_maiz_OLS ~ 1 + P_maiz_OLS + P_trigo_OLS', 24) for x in range(NSIMUL)],axis=1).T

Error cuadrático medio del coeficiente estimado del precio del maíz.

In [24]:
error_cuadratico_medio(simulM,-0.2,"P_maiz_OLS")

9204995.15344758

Error cuadrático medio del coeficiente estimado del precio del trigo.

In [25]:
error_cuadratico_medio(simulM,1.2,"P_trigo_OLS")

17770418.77834127

Error cuadrático medio del intercepto de la demanda de maíz.

In [34]:
error_cuadratico_medio(simulM,100,"Intercept")

44874046573.29519

Error cuadrático medio de la demanda de trigo.

In [26]:
simulTT = pd.concat([tsls_params('Q_trigo_OLS ~ 1 + P_maiz_OLS + P_trigo_OLS', 24) for x in range(NSIMUL)],axis=1).T

Error cuadrático medio del coeficiente estimado del precio del maíz.

In [27]:
error_cuadratico_medio(simulTT,1.1,"P_maiz_OLS")

952.91217604979

Error cuadrático medio del coeficiente estimado del precio del trigo.

In [28]:
error_cuadratico_medio(simulTT,-0.4,"P_trigo_OLS")

1274.9172488579459

Error cuadrático medio del intercepto de la demanda de trigo.

In [35]:
error_cuadratico_medio(simulTT,50,"Intercept")

2226659.930521876

Tabla comparativa de los estimadores.

|Estimación| MCO  | MC2E 
|---|---|---|
|Precio del maíz de la demanda de maíz   |0.0171   |9204995.15 
|Precio del trigo de la demanda de maíz   |0.2451   |17770418.77  
|Intercepto de la demanda de maíz        |3039.4272   |44874046573.2951
|Precio del maíz de la demanda de trigo   |0.2482   |952.91   
|Precio del trigo de la demanda de trigo  |0.0385   |1274.91
|Intercepto de la demanda de trigo        |1122.9055   |2226659.9305

Según el criterio del error cuadrático medio, ¿cuál de los dos estimadores (MCO o MC2E) es mejor para
estimar las ecuaciones de demanda?

A nivel del criterio del error cuadrático medio, el estimador MCO se consideraría mejor para estimar las ecuaciones de demanda pues posee el menor error cuadrático medio, a pesar de que es claro que estos estimadores están sesgados.

<h2>Pregunta 2</h2>

Falso. La cantidad de datos disponible no es relevante en este caso para poder estimar. Si una ecuación se encuentra "sub-identificada" en sí se dice que no está identificada y solo es posible estimar aquellas que están "sobre-identificada" o "exactamente identificada".

<h2>Pregunta 3</h2>

Falso. El método de estimación dependerá de la forma del sistema SUR. En caso de que el sistema cumple que sus regresores son los mismos para cada regresión entonces si se puede estimar por mínimos cuadrados ordinarios ecuación por ecuación, sino deberá ser estimado solo por mínimos cuadrados generalizados tomando los supuestos que el modelo SUR cumple con los supuestos del modelo generalizado de regresión lineal, de otra forma no tendría resultados que sean eficientes.

<h2>Pregunta 4</h2>

A=\begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}   B=\begin{bmatrix} 2 & -3 & & 4 \\ -4 & 3 & 2 & \end{bmatrix} I=\begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}


 $A\otimes B$

$\begin{bmatrix}
B & 2B \\
3B & 4B
\end{bmatrix}=\begin{bmatrix}
2 & -3 & 4 & 4 & 8 \\
-4 & 3 & -2 & -8 & -4 \\
6 & -9 & 12 & 8 & 16 \\
-12 & 9 & -6 & -16 & 8
\end{bmatrix}$


$B\otimes A$

$\begin{bmatrix} 2A & -3A & 4A \\
-4A & 3A & -2A \end{bmatrix}=\begin{bmatrix} 2 & 4 & -3 & -6 & 4 & 8 \\ 6 & 8 & -9 & -12 & 12 & 16 \\ -4 & -8 & 3 & 6 & -2 & -4 \\ -12 & -16 & 9 & 12 & -6 & -8 \end{bmatrix}
$

$A\otimes I$

$\begin{bmatrix}
1I & 2I \\
3I & 4I
\end{bmatrix}=\begin{bmatrix}
1 & 0 & 2 & 0 \\
0 & 1 & 0 & 2 \\
3 & 0 & 4 & 0 \\
0 & 3 & 0 & 4
\end{bmatrix}
$

$I\otimes A$


$\begin{bmatrix}
A & 0A \\
0A & A
\end{bmatrix}=\begin{bmatrix}
1 & 2 & 0 & 0 \\
3 & 4 & 0 & 0 \\
0 & 0 & 1 & 2 \\
0 & 0 & 3 & 4
\end{bmatrix}
$

$\begin{bmatrix}
4 & 0 & 1 & 0 \\
0 & 4 & 0 & 1 \\
2 & 0 & 3 & 0 \\
0 & 2 & 0 & 3
\end{bmatrix}^{-1}$


<center>$\begin{bmatrix}
4I & 1I \\
2I & 3I
\end{bmatrix}^{-1}
\qquad$ donde $\qquad$ $C=\begin{bmatrix}
4 & 1 \\
2 & 3
\end{bmatrix}
$
$\qquad$
</center>
<center>
así $\qquad$ $\begin{bmatrix}
4I & 1I \\
2I & 3I
\end{bmatrix}^{-1}= C^{-1}\otimes I^{-1}$</center>